<a href="https://colab.research.google.com/github/kirath2205/Final-Year-Project/blob/main/FGSM_attack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras,os
import tensorflow as tf
from keras.models import Sequential,Model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D , Dropout
import numpy as np
from tensorflow import keras
from keras.initializers import glorot_uniform
from keras import applications
from tensorflow.keras.applications.resnet50 import ResNet50 , preprocess_input
from keras import regularizers

# Install bleeding edge version of cleverhans
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.fast_gradient_method import fast_gradient_method

In [ ]:
cnn_model = keras.models.load_model('cnn_model.h5')
vgg_16 = keras.models.load_model('vgg16.h5')
resnet = keras.models.load_model('resnet.h5')

In [9]:
cifar_10 = tf.keras.datasets.cifar10.load_data()
(X_train, y_train), (X_test, y_test) = cifar_10
def pre_processing_cifar10(X_train,y_train,X_test,y_test,image_channel):
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')

  y_train = keras.utils.to_categorical(y_train)
  y_test = keras.utils.to_categorical(y_test)

  X_train = preprocess_input(X_train)
  X_test = preprocess_input(X_test)

  img_width = X_train[0].shape[0]
  img_height = X_train[0].shape[1]
  image_channel = image_channel

  input_shape=(img_width,img_height,image_channel) 
  return X_train,y_train,X_test,y_test
X_train,y_train,X_test,y_test = pre_processing_cifar10(X_train,y_train,X_test,y_test,3)

In [12]:

def make_predictions(model , image , true_value):

  labels = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

  true_label_index = -1
  for i in range(len(true_value)):

    if(true_value[i]==1):
      true_label_index = i
      break

  prediction = model.predict(image)[0]
  probability = float('-inf')
  predicted_label_index = -1

  for i in range(len(prediction)):

    if(prediction[i]>probability):
      probability = prediction[i]
      predicted_label_index = i

  predicted_label = labels[predicted_label_index]
  true_label = labels[true_label_index]

  print('Predicted Label -',predicted_label,' Probability -',probability)
  print('True Label -',true_label)

  return true_label_index == predicted_label_index


In [28]:
from keras.preprocessing.image import img_to_array

epsilon = 0.4
logits_cnn_model = tf.keras.Model(cnn_model.input,cnn_model.layers[-1].output)
total_images = 10
misclassification_indexes = []
for image_index in range(total_images):
  
  print('-------------------------------')
  print()
  print('Original Sample')
  image  = X_test[image_index]
  image = img_to_array(image)
  image = image.reshape(1, 32, 32, 3)
  true_value = y_test[image_index]
  original_prediction = make_predictions(cnn_model,image,true_value)
  adverserial_sample = fast_gradient_method(logits_cnn_model, image, epsilon, np.inf, targeted=False)
  print()
  print('Untargeted FGSM Adverserial Sample')
  adverserial_prediction = make_predictions(cnn_model , adverserial_sample , true_value)


  if(adverserial_prediction != original_prediction):
    misclassification_indexes.append(image_index)
  print()
  print('-------------------------------')

print(misclassification_indexes)

-------------------------------

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
Original Sample
Predicted Label - cat  Probability - 0.78703123
True Label - cat

Untargeted FGSM Adverserial Sample
Predicted Label - dog  Probability - 0.53274536
True Label - cat

-------------------------------
-------------------------------

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Original Sample
Predicted Label - ship  Probability - 0.91961604
True Label - ship

Untargeted FGSM Adverserial Sample
Predicted Label - ship  Probability - 0.57325405
True Label - ship

-------------------------------
-------------------------------

[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
Original Sample
Predicted Label - ship  Probability - 0.9972573
True Label - ship

Untargeted FGSM Adverserial Sample
Predicted Label - ship  Probability - 0.9804607
True Label - ship

-------------------------------
-------------------------------

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Original Sample
Predicted Label - airplane  Probability - 0.720281
True Label - airplane
